FORMERLY KNOWN AS BERTO (RIP)

In [ ]:
!pip install transformers


In [6]:
from google.colab import drive
drive.mount('/content/drive')
import os
import pandas as pd
import shutil
import torch
import tensorflow as tf
import numpy as np
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler, TensorDataset, random_split
from torch.nn import functional as F
import transformers
from transformers import BertTokenizer
from transformers import BertForSequenceClassification, AdamW, BertConfig
from transformers import BertTokenizer, BertForMaskedLM

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [7]:
# CHOOSE MODEL
vanilla = 'bert-base-uncased' # this is basic bert and has no saved weights
with_davidson = "bert_train"
with_davidson_parler = "bert_parler_train"
# with_davidson_4chan = "bert_4chan_train" # does not exist yet
saved_model = "saved_model"
big_parler = "bert_parler_large_train/checkpoint-110000"

# ASSIGN MODEL HERE
model_name = big_parler

# TRUE IF YOU ARE TESTING MODEL, FALSE IF YOU ARE TRAINING MODEL
# train_model = False
train_model = True

# UNCOMMENT BELOW BASED ON WHO IS RUNNING

# for joli (julia/oliver), assuming this works for both
prefix = "/content/drive/MyDrive/CS224N_Project"

# for jackson
# prefix = "/content/drive/MyDrive/Classes/Coterm/Winter Quarter/CS224N/CS224N_Project"

os.chdir(prefix)
output_dir = f'{prefix}/model_weights/{model_name}'

In [8]:
# If there's a GPU available...
if torch.cuda.is_available():    

    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: Tesla V100-SXM2-16GB


In [9]:
# Get the GPU device name.
device_name = tf.test.gpu_device_name()

# The device name should look like the following:
if device_name == '/device:GPU:0':
    print('Found GPU at: {}'.format(device_name))
else:
    raise SystemError('GPU device not found')

Found GPU at: /device:GPU:0


In [10]:
# Load the BERT tokenizer.
print('Loading BERT tokenizer...')
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

Loading BERT tokenizer...


In [ ]:


# Load BertForSequenceClassification, the pretrained BERT model with a single 
# linear classification layer on top. 
model_base = BertForMaskedLM.from_pretrained("bert-base-uncased", return_dict = True)

# Tell pytorch to run this model on the GPU.
model_base.cuda()

In [ ]:
model_ex = BertForMaskedLM.from_pretrained(None, state_dict="/content/drive/MyDrive/CS224N_Project/model_weights/state_dict_unzip/archive",
                                           config="/content/drive/MyDrive/CS224N_Project/model_weights/bert_train/config.json", return_dict = True)

# Tell pytorch to run this model on the GPU.
model_ex.cuda()

TEXT GENERATOR

In [ ]:
load_model_name = ""
if model_name != "bert-based-uncased":
  load_model_name = "model_weights/" + model_name
else:
  load_model_name = model_name

tokenizer = BertTokenizer.from_pretrained(load_model_name)
model_mask = BertForMaskedLM.from_pretrained(load_model_name, return_dict = True, pad_token_id=tokenizer.eos_token_id)
#model_mask = BertForMaskedLM.from_pretrained("bert-base-uncased",    return_dict = True)



# encode context the generation is conditioned on
input_ids = tokenizer.encode('I enjoy walking with my cute dog', return_tensors='tf')

# generate text until the output length (which includes the context length) reaches 50
greedy_output = model_mask.generate(input_ids, max_length=50)

print("Output:\n" + 100 * '-')
print(tokenizer.decode(greedy_output[0], skip_special_tokens=True))


In [ ]:
num_length = 10
text = "Dominion voting machines"
for i in range(num_length):
  text_to_gen = text + " " + tokenizer.mask_token
  input = tokenizer.encode_plus(text_to_gen, return_tensors = "pt")
  input.to(device)
  mask_index = torch.where(input["input_ids"][0] == tokenizer.mask_token_id)
  model_mask.cuda()
  output = model_mask(**input)
  logits = output.logits
  softmax = F.softmax(logits, dim = -1)
  mask_word = softmax[0, mask_index, :]
  top_10 = torch.topk(mask_word, 10, dim = 1)[1][0]
  #first = top_10[0]
  for token in top_10:
    word = tokenizer.decode([token])
    if word != "!" and word != "." and word != "," and word != "?" and word != ":" and word != ")":
      new_sentence = text_to_gen.replace(tokenizer.mask_token, word)
      break
  text = new_sentence
  print(new_sentence)
print(text)

Dominion voting machines too
Dominion voting machines too much
Dominion voting machines too much fraud
Dominion voting machines too much fraud ##s
Dominion voting machines too much fraud ##s ##j
Dominion voting machines too much fraud ##s ##j #
Dominion voting machines too much fraud ##s ##j # d
Dominion voting machines too much fraud ##s ##j # d #
Dominion voting machines too much fraud ##s ##j # d # d
Dominion voting machines too much fraud ##s ##j # d # d #
Dominion voting machines too much fraud ##s ##j # d # d #


In [ ]:
text_to_gen = "January 6th is " + tokenizer.mask_token
input = tokenizer.encode_plus(text_to_gen, return_tensors = "pt")
input.to(device)
mask_index = torch.where(input["input_ids"][0] == tokenizer.mask_token_id)
model_mask.cuda()
output = model_mask(**input)
logits = output.logits
softmax = F.softmax(logits, dim = -1)
mask_word = softmax[0, mask_index, :]
top_10 = torch.topk(mask_word, 10, dim = 1)[1][0]
for token in top_10:
  word = tokenizer.decode([token])
  print(word)

coming
over
now
time
friday
happening
wednesday
why
done
ridiculous


In [ ]:
# Get all of the model's parameters as a list of tuples.
params = list(model.named_parameters())

print('The BERT model has {:} different named parameters.\n'.format(len(params)))

print('==== Embedding Layer ====\n')

for p in params[0:5]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

print('\n==== First Transformer ====\n')

for p in params[5:21]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

print('\n==== Output Layer ====\n')

for p in params[-4:]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

The BERT model has 201 different named parameters.

==== Embedding Layer ====

bert.embeddings.word_embeddings.weight                  (30522, 768)
bert.embeddings.position_embeddings.weight                (512, 768)
bert.embeddings.token_type_embeddings.weight                (2, 768)
bert.embeddings.LayerNorm.weight                              (768,)
bert.embeddings.LayerNorm.bias                                (768,)

==== First Transformer ====

bert.encoder.layer.0.attention.self.query.weight          (768, 768)
bert.encoder.layer.0.attention.self.query.bias                (768,)
bert.encoder.layer.0.attention.self.key.weight            (768, 768)
bert.encoder.layer.0.attention.self.key.bias                  (768,)
bert.encoder.layer.0.attention.self.value.weight          (768, 768)
bert.encoder.layer.0.attention.self.value.bias                (768,)
bert.encoder.layer.0.attention.output.dense.weight        (768, 768)
bert.encoder.layer.0.attention.output.dense.bias              (

In [ ]:
# Note: AdamW is a class from the huggingface library (as opposed to pytorch) 
# I believe the 'W' stands for 'Weight Decay fix"
optimizer = AdamW(model.parameters(),
                  lr = 2e-5, # args.learning_rate - default is 5e-5, our notebook had 2e-5
                  eps = 1e-8 # args.adam_epsilon  - default is 1e-8.
                )

In [ ]:
from transformers import get_linear_schedule_with_warmup

# Number of training epochs. The BERT authors recommend between 2 and 4. 
# We chose to run for 4, but we'll see later that this may be over-fitting the
# training data.
epochs = 4

# Total number of training steps is [number of batches] x [number of epochs]. 
# (Note that this is not the same as the number of training samples).
total_steps = len(train_dataloader) * epochs

# Create the learning rate scheduler.
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0, # Default value in run_glue.py
                                            num_training_steps = total_steps)

In [ ]:
import numpy as np

# Function to calculate the accuracy of our predictions vs labels
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

In [ ]:
import time
import datetime

def format_time(elapsed):
    '''
    Takes a time in seconds and returns a string hh:mm:ss
    '''
    # Round to the nearest second.
    elapsed_rounded = int(round((elapsed)))
    
    # Format as hh:mm:ss
    return str(datetime.timedelta(seconds=elapsed_rounded))

In [ ]:
import random
import numpy as np

# This training code is based on the `run_glue.py` script here:
# https://github.com/huggingface/transformers/blob/5bfcd0485ece086ebcbed2d008813037968a9e58/examples/run_glue.py#L128

# Set the seed value all over the place to make this reproducible.
seed_val = 42

random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

# We'll store a number of quantities such as training and validation loss, 
# validation accuracy, and timings.
training_stats = []

# Measure the total training time for the whole run.
total_t0 = time.time()

# For each epoch...
for epoch_i in range(0, epochs):
    
    # ========================================
    #               Training
    # ========================================
    
    # Perform one full pass over the training set.

    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    # Measure how long the training epoch takes.
    t0 = time.time()

    # Reset the total loss for this epoch.
    total_train_loss = 0

    # Put the model into training mode. Don't be mislead--the call to 
    # `train` just changes the *mode*, it doesn't *perform* the training.
    # `dropout` and `batchnorm` layers behave differently during training
    # vs. test (source: https://stackoverflow.com/questions/51433378/what-does-model-train-do-in-pytorch)
    model.train()

    # For each batch of training data...
    for step, batch in enumerate(train_dataloader):

        # Progress update every 40 batches.
        if step % 40 == 0 and not step == 0:
            # Calculate elapsed time in minutes.
            elapsed = format_time(time.time() - t0)
            
            # Report progress.
            print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))
        if step % 2000 == 0:
          model.save_pretrained("/content/drive/MyDrive/CS224N_Project/")
        # Unpack this training batch from our dataloader. 
        #
        # As we unpack the batch, we'll also copy each tensor to the GPU using the 
        # `to` method.
        #
        # `batch` contains three pytorch tensors:
        #   [0]: input ids 
        #   [1]: attention masks
        #   [2]: labels 
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)

        # Always clear any previously calculated gradients before performing a
        # backward pass. PyTorch doesn't do this automatically because 
        # accumulating the gradients is "convenient while training RNNs". 
        # (source: https://stackoverflow.com/questions/48001598/why-do-we-need-to-call-zero-grad-in-pytorch)
        model.zero_grad()        

        # Perform a forward pass (evaluate the model on this training batch).
        # The documentation for this `model` function is here: 
        # https://huggingface.co/transformers/v2.2.0/model_doc/bert.html#transformers.BertForSequenceClassification
        # It returns different numbers of parameters depending on what arguments
        # arge given and what flags are set. For our useage here, it returns
        # the loss (because we provided labels) and the "logits"--the model
        # outputs prior to activation.
        
        # loss, logits = model(b_input_ids, 
        #                      token_type_ids=None, 
        #                      attention_mask=b_input_mask, 
        #                      labels=b_labels)
        output = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask, labels=b_labels)
        loss = output.loss
        logits = output.logits
        # Accumulate the training loss over all of the batches so that we can
        # calculate the average loss at the end. `loss` is a Tensor containing a
        # single value; the `.item()` function just returns the Python value 
        # from the tensor.
        total_train_loss += loss.item()

        # Perform a backward pass to calculate the gradients.
        loss.backward()

        # Clip the norm of the gradients to 1.0.
        # This is to help prevent the "exploding gradients" problem.
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        # Update parameters and take a step using the computed gradient.
        # The optimizer dictates the "update rule"--how the parameters are
        # modified based on their gradients, the learning rate, etc.
        optimizer.step()

        # Update the learning rate.
        scheduler.step()

    # Calculate the average loss over all of the batches.
    avg_train_loss = total_train_loss / len(train_dataloader)            
    
    # Measure how long this epoch took.
    training_time = format_time(time.time() - t0)

    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epcoh took: {:}".format(training_time))
        
    # ========================================
    #               Validation
    # ========================================
    # After the completion of each training epoch, measure our performance on
    # our validation set.

    print("")
    print("Running Validation...")

    t0 = time.time()

    # Put the model in evaluation mode--the dropout layers behave differently
    # during evaluation.
    model.eval()

    # Tracking variables 
    total_eval_accuracy = 0
    total_eval_loss = 0
    nb_eval_steps = 0

    # Evaluate data for one epoch
    for batch in validation_dataloader:
        
        # Unpack this training batch from our dataloader. 
        #
        # As we unpack the batch, we'll also copy each tensor to the GPU using 
        # the `to` method.
        #
        # `batch` contains three pytorch tensors:
        #   [0]: input ids 
        #   [1]: attention masks
        #   [2]: labels 
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)
        
        # Tell pytorch not to bother with constructing the compute graph during
        # the forward pass, since this is only needed for backprop (training).
        with torch.no_grad():        

            # Forward pass, calculate logit predictions.
            # token_type_ids is the same as the "segment ids", which 
            # differentiates sentence 1 and 2 in 2-sentence tasks.
            # The documentation for this `model` function is here: 
            # https://huggingface.co/transformers/v2.2.0/model_doc/bert.html#transformers.BertForSequenceClassification
            # Get the "logits" output by the model. The "logits" are the output
            # values prior to applying an activation function like the softmax.
            output = model(b_input_ids, 
                                   token_type_ids=None, 
                                   attention_mask=b_input_mask,
                                   labels=b_labels)
            loss = output.loss
            logits = output.logits
        # Accumulate the validation loss.
        total_eval_loss += loss.item()

        # Move logits and labels to CPU
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()

        # Calculate the accuracy for this batch of test sentences, and
        # accumulate it over all batches.
        total_eval_accuracy += flat_accuracy(logits, label_ids)
        

    # Report the final accuracy for this validation run.
    avg_val_accuracy = total_eval_accuracy / len(validation_dataloader)
    print("  Accuracy: {0:.2f}".format(avg_val_accuracy))

    # Calculate the average loss over all of the batches.
    avg_val_loss = total_eval_loss / len(validation_dataloader)
    
    # Measure how long the validation run took.
    validation_time = format_time(time.time() - t0)
    
    print("  Validation Loss: {0:.2f}".format(avg_val_loss))
    print("  Validation took: {:}".format(validation_time))

    # Record all statistics from this epoch.
    training_stats.append(
        {
            'epoch': epoch_i + 1,
            'Training Loss': avg_train_loss,
            'Valid. Loss': avg_val_loss,
            'Valid. Accur.': avg_val_accuracy,
            'Training Time': training_time,
            'Validation Time': validation_time
        }
    )

print("")
print("Training complete!")

print("Total training took {:} (h:mm:ss)".format(format_time(time.time()-total_t0)))


======== Epoch 1 / 4 ========
Training...
  Batch    40  of  1,240.    Elapsed: 0:00:27.
  Batch    80  of  1,240.    Elapsed: 0:00:47.
  Batch   120  of  1,240.    Elapsed: 0:01:06.
  Batch   160  of  1,240.    Elapsed: 0:01:26.
  Batch   200  of  1,240.    Elapsed: 0:01:46.
  Batch   240  of  1,240.    Elapsed: 0:02:05.
  Batch   280  of  1,240.    Elapsed: 0:02:25.
  Batch   320  of  1,240.    Elapsed: 0:02:45.
  Batch   360  of  1,240.    Elapsed: 0:03:04.
  Batch   400  of  1,240.    Elapsed: 0:03:24.
  Batch   440  of  1,240.    Elapsed: 0:03:43.
  Batch   480  of  1,240.    Elapsed: 0:04:03.
  Batch   520  of  1,240.    Elapsed: 0:04:23.
  Batch   560  of  1,240.    Elapsed: 0:04:42.
  Batch   600  of  1,240.    Elapsed: 0:05:02.
  Batch   640  of  1,240.    Elapsed: 0:05:22.
  Batch   680  of  1,240.    Elapsed: 0:05:41.
  Batch   720  of  1,240.    Elapsed: 0:06:01.
  Batch   760  of  1,240.    Elapsed: 0:06:21.
  Batch   800  of  1,240.    Elapsed: 0:06:40.
  Batch   840  of